In [1]:
import boto3
import pandas as pd
from io import StringIO
import io
import json
import warnings
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from datetime import datetime

warnings.filterwarnings("ignore")

In [2]:
with open('conf.json' , 'r') as conf:
    config = json.loads(conf.read())
    dynamic_part = f"{config['client']}/{config['year']}/{config['month']}/{config['destination_configuration']['suffix']}"

if config is None:
    print('ERROR: Problem parsing config file')
    sys.exit()

In [3]:
def select_open_risk_records(config):
    rds_secret = get_secret('dev/rds')
    host = rds_secret['host']
    username = rds_secret['username']
    password = rds_secret['password']
    engine = rds_secret['engine']
    port = rds_secret['port']
    db_name = rds_secret['dbname']
    schema = config['rds']['schema_name']
    table_name = config['rds']['risk_gaps_table_name']

    if engine == 'postgres':
        engine = 'postgresql'

    engine_creds = f"{engine}://{username}:{password}@{host}:{port}/{db_name}"
    engine = create_engine(engine_creds, connect_args={'options': f'-csearch_path={schema}'})
    connection = engine.connect()
    # Construct the SQL query based on the provided month and year
    query = text(f"SELECT * FROM {table_name} where client = '{config['client']}' AND year = '{config['year']}' AND month = '{config['month']}' ")

    # Execute the query and load the results into a DataFrame
    open_risk_gaps = pd.read_sql(query, connection)
    open_risk_gaps['clientmemberid'] = open_risk_gaps['clientmemberid'].astype(str)
    open_risk_gaps['ruleidentifier'] = open_risk_gaps['ruleidentifier'].astype(str)
    open_risk_gaps.rename(columns={'ruleidentifier':'hcc_code'}, inplace=True)
    
    open_risk_gaps['Appointment Date'] = ''
    open_risk_gaps['Gap Tracker'] = ''
    open_risk_gaps['Notes'] = ''

    # Close the database connection
    connection.close()
    engine.dispose()

    return open_risk_gaps


In [4]:

def get_secret(secret_name , region_name='us-east-2'):
    # Create a Secrets Manager client
    boto3_session = get_boto3_session()
    client = boto3_session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except Exception as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        print(f"Error getting SecretString from Secrets Manager: {str(e)}")
        sys.exit()

    # Decrypts secret using the associated KMS key.
    secret = get_secret_value_response['SecretString']
    secret = json.loads(secret)
    return secret

In [5]:
def get_boto3_session():
    aws_profile = 'paradocs'
    try:
        session = boto3.Session(profile_name=aws_profile)
    except Exception as e:
        print('Error loading paradocs profile')
        session = boto3.Session()
    return session

In [6]:
def read_CodeX_files(s3_client, bucket, prefix):
    all_dataframes = []
    paginator = s3_client.get_paginator('list_objects_v2')

    for page in paginator.paginate(Bucket=bucket_name, Prefix=prefix):    
        # Extract keys from the current page
        keys = [obj['Key'] for obj in page.get('Contents', [])]
        # Skip if there are no keys
        if not keys:
            continue

        for key in keys:
            if not key.endswith('.csv'):
                continue
            response = s3_client.get_object(Bucket=bucket_name, Key=key)
            csv_content = response['Body'].read().decode('utf-8')
            df = pd.read_csv(StringIO(csv_content))
            all_dataframes.append(df)
            print('Iteration#: ', len(all_dataframes))
            if len(all_dataframes) > 500: break

    consolidated_df = pd.concat(all_dataframes, ignore_index=True)
    consolidated_df['clientMemberId'] = consolidated_df['clientMemberId'].astype(str)
    consolidated_df.rename(columns={'clientMemberId':'clientmemberid'}, inplace=True)
    consolidated_df = consolidated_df[consolidated_df['codeType'] == 'ICD10']
    consolidated_df['ICD_Code'] = consolidated_df['ICD_Code'].str.replace('.', '')
    consolidated_df = consolidated_df.drop_duplicates()
    consolidated_df = consolidated_df.sort_values('Appointment_Date')
    consolidated_df = consolidated_df.drop_duplicates(subset=['PD_Pin', 'clientmemberid', 'ICD_Code', 'codeType', 'ICD_Name'], keep='last')
    return consolidated_df

In [7]:
def map_icd10_to_hcc(patients_df, icd10_to_hcc_df):
    merged_df = pd.merge(patients_df, icd10_to_hcc_df, how='left', on='ICD_Code')
    merged_df.loc[merged_df['hcc_code'] == 'NA', 'hcc_code'] = np.NaN
    merged_df['hcc_code'] = merged_df['hcc_code'].astype(str)
    merged_df['hcc_code'] = merged_df['hcc_code'].str.split('.').str[0]

    return merged_df


In [8]:
def load_hcc_lookup_table(config):
    bucket = config["midyear_final_bucket"]
    midyear_final_filename = config["midyear_final_filename"]
    midyear_final_column_names = config["midyear_final_column_names"]
    midyear_file_key = f'{midyear_final_filename}'
    boto3_session = get_boto3_session()
    s3_client = boto3_session.client('s3')
    obj = s3_client.get_object(Bucket=bucket, Key=midyear_file_key)
    midyear_final_df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    hcc_lookup_df = midyear_final_df[midyear_final_column_names]
    hcc_lookup_df.columns = ['ICD_Code', 'hcc_code']
    hcc_lookup_df = hcc_lookup_df.fillna('NA')
    hcc_lookup_df = hcc_lookup_df.drop_duplicates()
    return hcc_lookup_df


In [9]:
def update_appointment_dates(patient_dict):
    rds_secret = get_secret('dev/rds')
    timestamp = str(datetime.now())
    host = rds_secret['host']
    username = rds_secret['username']
    password = rds_secret['password']
    engine = rds_secret['engine']
    port = rds_secret['port']
    db_name = rds_secret['dbname']
    schema = config['rds']['schema_name']
    table_name = config['rds']['risk_gaps_table_name']

    if engine == 'postgres':
        engine = 'postgresql'

    engine_creds = f"{engine}://{username}:{password}@{host}:{port}/{db_name}"
    engine = create_engine(engine_creds, connect_args={'options': f'-csearch_path={schema}'})
    # Define your SQLAlchemy engine

    try:
        # Iterate through the dictionary and update the database
        with engine.connect() as connection:
            for uniquemembergapid, inner_dict in patient_dict.items():
                # Construct the raw SQL update statement
                update_query = f"""UPDATE {table_name} SET "Appointment Date" = {inner_dict.get('appointment_date')}, pdpin = '{inner_dict.get('pdpin')}', "Gap Tracker" = 'Assessed and Present', updated_at = '{timestamp}' WHERE uniquemembergapid = '{uniquemembergapid}';"""
                update_query = text(update_query)
                # Execute the raw SQL update statement
                connection.execute(update_query)
            connection.commit()

    except Exception as e:
        # Handle exceptions (rollback, log, etc.)
        print(f"Error: {e}")


In [10]:
boto3_session = get_boto3_session()

s3_client = boto3_session.client('s3', region_name='us-east-2')

bucket_name = config['post-processing']['bucket']

prefix = f"{config['client']}/{config['year']}/{config['month']}/codex/"

In [11]:
patients_df = read_CodeX_files(s3_client, bucket_name, prefix)

Iteration#:  1
Iteration#:  2
Iteration#:  3
Iteration#:  4
Iteration#:  5
Iteration#:  6
Iteration#:  7
Iteration#:  8
Iteration#:  9
Iteration#:  10
Iteration#:  11
Iteration#:  12
Iteration#:  13
Iteration#:  14
Iteration#:  15
Iteration#:  16
Iteration#:  17
Iteration#:  18
Iteration#:  19
Iteration#:  20
Iteration#:  21
Iteration#:  22
Iteration#:  23
Iteration#:  24
Iteration#:  25
Iteration#:  26
Iteration#:  27
Iteration#:  28
Iteration#:  29
Iteration#:  30
Iteration#:  31
Iteration#:  32
Iteration#:  33
Iteration#:  34
Iteration#:  35
Iteration#:  36
Iteration#:  37
Iteration#:  38
Iteration#:  39
Iteration#:  40
Iteration#:  41
Iteration#:  42
Iteration#:  43
Iteration#:  44
Iteration#:  45
Iteration#:  46
Iteration#:  47
Iteration#:  48
Iteration#:  49
Iteration#:  50
Iteration#:  51
Iteration#:  52
Iteration#:  53
Iteration#:  54
Iteration#:  55
Iteration#:  56
Iteration#:  57
Iteration#:  58
Iteration#:  59
Iteration#:  60
Iteration#:  61
Iteration#:  62
Iteration#:  63
I

In [29]:
patients_df

,Appointment_Date,PD_Pin,clientmemberid,ICD_Code,codeType,ICD_Name
636,20230103,2708c5e5c277bb19ba29961813e2fd92ed79d03b578551...,nan,J0190,ICD10,"Acute sinusitis, unspecified"
634,20230103,2708c5e5c277bb19ba29961813e2fd92ed79d03b578551...,nan,H9209,ICD10,"Otalgia, unspecified ear"
2138,20230104,addc151ddc28aece3cb42ea8e9d1d2261a0ba4ff7f0dff...,nan,E782,ICD10,Mixed hyperlipidemia
122,20230105,0578302dfe71f0075c09f94ffdb9f101a683c98ba649c0...,nan,K219,ICD10,Gastro-esophageal reflux disease without esoph...
123,20230105,0578302dfe71f0075c09f94ffdb9f101a683c98ba649c0...,nan,R0602,ICD10,Shortness of breath
...,...,...,...,...,...,...
1984,20231107,a589c768e0bba73093a7be81cce37bb0305a9f39a0f5c1...,nan,I700,ICD10,Atherosclerosis of aorta
1983,20231107,a589c768e0bba73093a7be81cce37bb0305a9f39a0f5c1...,nan,E119,ICD10,Type 2 diabetes mellitus without complications
1982,20231107,a589c768e0bba73093a7be81cce37bb0305a9f39a0f5c1...,nan,I517,ICD10,Cardiomegaly
2403,20231107,cbfa6a19fdb21507968a69c063af97caef2480ae430a71...,nan,K5720,ICD10,Diverticulitis of large intestine with perfora...


In [34]:
# patients_df[patients_df['uniquememberid'] == 'eb2bd9a4-a3b8-4151-8a4c-0d87b13bae5c']

In [13]:
hcc_lookup = load_hcc_lookup_table(config)

In [26]:
hcc_lookup

,ICD_Code,hcc_code
0,A0103,115.0
1,A0104,39.0
2,A0105,39.0
3,A021,2.0
4,A0222,115.0
...,...,...
11523,NA,NA
11524,Note:,NA
11525,If an ICD-10 code includes clinical concepts c...,NA
11528,Output: icd10_udxgv0123_v21v22v24v05v08_p1.xlsx,NA


In [27]:
hcc_lookup[hcc_lookup['ICD_Code'] == 'M069']

,ICD_Code,hcc_code
6508,M069,40.0


In [15]:
patients_hcc_df = map_icd10_to_hcc(patients_df, hcc_lookup)

In [20]:
patients_hcc_df[patients_hcc_df['hcc_code']!='nan']

,Appointment_Date,PD_Pin,clientmemberid,ICD_Code,codeType,ICD_Name,hcc_code
24,20230112,5ef29ceb7d97a4c61e36f46fc2539e33c5947c6ab80e31...,nan,F3489,ICD10,Other specified persistent mood disorders,59
32,20230117,857ebd23a25bc784e946c993811a4bf574fa79d82c0a5c...,nan,J9601,ICD10,Acute respiratory failure with hypoxia,84
34,20230117,857ebd23a25bc784e946c993811a4bf574fa79d82c0a5c...,nan,I4891,ICD10,Unspecified atrial fibrillation,96
49,20230123,3dc26321b3fb4a5eeffe5642d81b4eaf34ecd713e4da5a...,nan,I739,ICD10,"Peripheral vascular disease, unspecified",108
59,20230124,3d471bc2d87d1d37d93746a86e5e02efc053b0b7bcaa30...,nan,I70203,ICD10,Unspecified atherosclerosis of native arteries...,108
...,...,...,...,...,...,...,...
1585,20231107,d912ef3cc1cbb87a476be7140705beaa96f5c849ef3b74...,nan,E6601,ICD10,Morbid (severe) obesity due to excess calories,22
1586,20231107,d912ef3cc1cbb87a476be7140705beaa96f5c849ef3b74...,nan,Z6842,ICD10,"Body mass index [BMI] 45.0-49.9, adult",22
1587,20231107,d912ef3cc1cbb87a476be7140705beaa96f5c849ef3b74...,nan,F39,ICD10,Unspecified mood [affective] disorder,59
1595,20231107,a589c768e0bba73093a7be81cce37bb0305a9f39a0f5c1...,nan,I700,ICD10,Atherosclerosis of aorta,108


In [35]:
open_risk_gaps = select_open_risk_records(config)

In [52]:
open_risk_gaps[open_risk_gaps['uniquememberid'] == 'COD172371134C']

,clientmemberid,uniquememberid,uniquemembergapid,hcc_code,gapdescription,gapreason,uniqueprovidergroupid,Submitted Date,month,year,client,Appointment Date,Gap Tracker,pdpin,created_at,updated_at,Notes
8,H50641661,COD172371134C,7988ea5e-e154-3243-9519-75491687858d,108,Vascular Disease; consider location & underlyi...,PREVIOUSLY CODED - VASCULAR DISEASE (E11.51); ...,20560761.0,None,11,2023,cypress-physicians-association,,,d8cae3a313b43c6d242b8375c5be64e90f3bc4c38e1d79...,2023-11-24 14:38:16.611689,2023-11-24 14:38:34.298541,
455,H50641661,COD172371134C,0eed6793-5518-311b-b6d4-704554cff23a,111,"COPD (J41-J44, J98)",PREVIOUSLY CODED - COPD (J42),20560761.0,None,11,2023,cypress-physicians-association,,,d8cae3a313b43c6d242b8375c5be64e90f3bc4c38e1d79...,2023-11-24 14:38:16.611689,2023-11-24 14:38:16.611689,


In [47]:
a

['538a03bfd2e121c9a7fecf69430d5ad540d9319a1072469e16c637eb35437f7f']

In [21]:
# patients_hcc_df['hcc_code'] = patients_hcc_df['hcc_code'].astype(str)
# patients_hcc_df['clientmemberid'] = patients_hcc_df['clientmemberid'].astype(str)

# open_risk_gaps['hcc_code'] = open_risk_gaps['hcc_code'].astype(str)
# open_risk_gaps['clientmemberid'] = open_risk_gaps['clientmemberid'].astype(str)

In [27]:
matched_df = pd.merge(patients_hcc_df, open_risk_gaps, how='inner', on=['clientmemberid', 'hcc_code'])

In [28]:
matched_df = matched_df.drop_duplicates(subset=['uniquememberid'], keep='last')

In [29]:
matched_dict = matched_df[['uniquemembergapid', 'Appointment_Date', 'PD_Pin', 'uniqueprovidergroupid']].set_index('uniquemembergapid').to_dict()

In [30]:
matched_dict

{'Appointment_Date': {'f0a8c4a9-89dc-3c6e-af90-c6440b084004': 20230307,
  '1eb3bd12-9b62-33c3-bd37-0c485cebab85': 20230411,
  'a94ce43b-b01b-360d-a2d2-f7627eecc386': 20231005,
  '8c613faf-2b83-37dd-b23b-d7cc00243518': 20231103,
  'ac6a0965-6233-3703-abad-fa1d2f77ed02': 20231017,
  '45781449-8ab9-3288-8a32-84ab3efee875': 20231101},
 'PD_Pin': {'f0a8c4a9-89dc-3c6e-af90-c6440b084004': '1cb3f2b0c448efab9d91aa22734e9cb1b1e994e9aa1c4df0b4fe311596727faa',
  '1eb3bd12-9b62-33c3-bd37-0c485cebab85': '0b4d426ecd204dc22757025bb8de43d593d1157c4b915b2e8f267a5a8de0323a',
  'a94ce43b-b01b-360d-a2d2-f7627eecc386': '20aa3b167e1a2a8770103f4aaef48c52f152060d3fb08a141262b3a8768e0d1c',
  '8c613faf-2b83-37dd-b23b-d7cc00243518': '0a0542b6020bf14dfcedc3e798c4a17b212b2e28caf4821103e3a0a60a81af8b',
  'ac6a0965-6233-3703-abad-fa1d2f77ed02': '0741958bae9f0a2bea3daec25e1bad55da894dbad8790f71fd26c3215e727132',
  '45781449-8ab9-3288-8a32-84ab3efee875': '24a3f7eda63b068bd7965573cac0eb1fde7c4df48c41046b4105aa82686401b4

In [31]:
print(matched_dict['Appointment_Date'])

{'f0a8c4a9-89dc-3c6e-af90-c6440b084004': 20230307, '1eb3bd12-9b62-33c3-bd37-0c485cebab85': 20230411, 'a94ce43b-b01b-360d-a2d2-f7627eecc386': 20231005, '8c613faf-2b83-37dd-b23b-d7cc00243518': 20231103, 'ac6a0965-6233-3703-abad-fa1d2f77ed02': 20231017, '45781449-8ab9-3288-8a32-84ab3efee875': 20231101}


In [34]:
new_dict = {}
for k,v in matched_dict['Appointment_Date'].items():
    new_dict[k] = {
        "pdpin": matched_dict['PD_Pin'][k],
        "appointment_date": v,
    "uniqueprovidergroupid": str(matched_dict['uniqueprovidergroupid'][k]).split('.')[0]}

In [36]:
print('--------DICT FOR NEXT STEP--------')
print('\n\n\n')
print('\n\n\n')
print(new_dict)
print('\n\n\n')
print('\n\n\n')

--------DICT FOR NEXT STEP--------








{'f0a8c4a9-89dc-3c6e-af90-c6440b084004': {'pdpin': '1cb3f2b0c448efab9d91aa22734e9cb1b1e994e9aa1c4df0b4fe311596727faa', 'appointment_date': 20230307, 'uniqueprovidergroupid': '825286393'}, '1eb3bd12-9b62-33c3-bd37-0c485cebab85': {'pdpin': '0b4d426ecd204dc22757025bb8de43d593d1157c4b915b2e8f267a5a8de0323a', 'appointment_date': 20230411, 'uniqueprovidergroupid': '825286393'}, 'a94ce43b-b01b-360d-a2d2-f7627eecc386': {'pdpin': '20aa3b167e1a2a8770103f4aaef48c52f152060d3fb08a141262b3a8768e0d1c', 'appointment_date': 20231005, 'uniqueprovidergroupid': '825286393'}, '8c613faf-2b83-37dd-b23b-d7cc00243518': {'pdpin': '0a0542b6020bf14dfcedc3e798c4a17b212b2e28caf4821103e3a0a60a81af8b', 'appointment_date': 20231103, 'uniqueprovidergroupid': '825286393'}, 'ac6a0965-6233-3703-abad-fa1d2f77ed02': {'pdpin': '0741958bae9f0a2bea3daec25e1bad55da894dbad8790f71fd26c3215e727132', 'appointment_date': 20231017, 'uniqueprovidergroupid': '825286393'}, '45781449-8ab9-3288-8

In [58]:
update_appointment_dates(new_dict)
print('done')

UPDATE open_risk_gaps SET "Appointment Date" = 20230307, pdpin = '1cb3f2b0c448efab9d91aa22734e9cb1b1e994e9aa1c4df0b4fe311596727faa', "Gap Tracker" = 'Assessed and Present', updated_at = '2023-11-16 01:50:05.133797' WHERE uniquemembergapid = 'f0a8c4a9-89dc-3c6e-af90-c6440b084004';
UPDATE open_risk_gaps SET "Appointment Date" = 20230411, pdpin = '0b4d426ecd204dc22757025bb8de43d593d1157c4b915b2e8f267a5a8de0323a', "Gap Tracker" = 'Assessed and Present', updated_at = '2023-11-16 01:50:05.133797' WHERE uniquemembergapid = '1eb3bd12-9b62-33c3-bd37-0c485cebab85';
UPDATE open_risk_gaps SET "Appointment Date" = 20231005, pdpin = '20aa3b167e1a2a8770103f4aaef48c52f152060d3fb08a141262b3a8768e0d1c', "Gap Tracker" = 'Assessed and Present', updated_at = '2023-11-16 01:50:05.133797' WHERE uniquemembergapid = 'a94ce43b-b01b-360d-a2d2-f7627eecc386';
UPDATE open_risk_gaps SET "Appointment Date" = 20231107, pdpin = '2a5b0216c2feed1994836acdf47793e70df7ec437ade3028c303b232cca1554f', "Gap Tracker" = 'Assesse

# ----------------DONE -------------------

In [38]:
# Save the dictionary to a JSON file
file_path = 'matched_patients_dict.json'
with open(file_path, 'w') as json_file:
    json.dump(new_dict, json_file)

print(f'Data saved to {file_path}')

Data saved to matched_patients_dict.json
